# Structured Output

In [1]:
from devtools import debug

!export PYTHONPATH=":./python"

### Method 1 : provide instruction in the prompt

In [16]:
"""
The usual "tell me a joke" LLM call.
"""

from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# IMPORTANT : select Pydantic V1
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt


class Joke(BaseModel):
    the_joke: str = Field(description="a good joke")
    explanation: str = Field(description="explain why it's funny")
    rate: float = Field(description="rate how the joke is funny between 0 and 5")


parser = PydanticOutputParser(pydantic_object=Joke)

prompt_with_format = """
    tell me  a joke on {topic}     
    --- 
    {format_instructions}"""

structured_prompt = def_prompt(user=prompt_with_format).partial(
    format_instructions=parser.get_format_instructions(),
)

parser = PydanticOutputParser(pydantic_object=Joke)
structured_joke = structured_prompt | get_llm() | parser

r = structured_joke.invoke({"topic": "cat"})
debug(r)

2024-06-27 16:18:13.760 | INFO     | python.ai_core.llm:get_llm:395 - get LLM : gpt_35_edenai - configurable: True
2024-06-27 16:18:14.140 | INFO     | python.ai_core.llm:get_configurable:351 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


ConnectionError: HTTPSConnectionPool(host='api.edenai.run', port=443): Max retries exceeded with url: /v2/text/chat (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fc7551d1840>: Failed to resolve 'api.edenai.run' ([Errno -3] Temporary failure in name resolution)"))

In [9]:
# You can have a look at the generated prompt:
print(structured_prompt.invoke({"topic": "cat"}).messages[0].content)


tell me  a joke on cat;     
--- 
    The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"the_joke": {"description": "a good joke", "title": "The Joke", "type": "string"}, "explanation": {"description": "explain why it's funny", "title": "Explanation", "type": "string"}, "rate": {"description": "rate how the joke is funny between 0 and 5", "title": "Rate", "type": "number"}}, "required": ["the_joke", "explanation", "rate"]}
```



### Model 2 : Use "with_structured_output"  (bases on function calls)

In [15]:
prompt = "tell me  a joke on {topic}"

chain = def_prompt(prompt) | get_llm().with_structured_output(Joke)
debug(chain.invoke(({"topic": "cat"})))

2024-06-27 16:10:02.009 | INFO     | python.ai_core.llm:get_llm:395 - get LLM : gpt_35_openai - configurable: True
2024-06-27 16:10:02.331 | INFO     | python.ai_core.llm:get_configurable:351 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


/tmp/ipykernel_40905/439611710.py:8 <module>
    chain.invoke(({"topic": "cat"})): Joke(
        the_joke='Why was the cat sitting on the computer?',
        explanation='Because it wanted to keep an eye on the mouse!',
    ) (Joke)


Joke(the_joke='Why was the cat sitting on the computer?', explanation='Because it wanted to keep an eye on the mouse!')